In [308]:
import numpy as np
import pandas as pd
# import sklearn.datasets as data

In [309]:
df = pd.read_csv("GroupExam2024_data_and_documentation/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep=';', decimal=',')
df.head()
print(df.shape)


(305434, 45)


In [310]:
#Sile ut informasjon:
df = df[(df['Art - FDIR (kode)'] == 1022) | (df['Art - FDIR (kode)'] == 1027)] # Fiske som fanger hyse eller torsk. "751 - lodde, 1022 Torsk"
area = [7, 8, 28, 30, 40]
df = df[(df['Redskap FDIR (kode)'] == 51) & (df['Hovedområde start (kode)'].isin(area))] # Fiske med bunntrål - 51, snurrevad - 11
df['Bruttotonnasje Kombinert'] = df['Bruttotonnasje 1969'].fillna(df['Bruttotonnasje annen']) # Slår sammen kolonnene.
# Dropper unødvendig informasjon. Tabellen inneholder masse duplikater. som egen kolonne for navn og for kode
# Forskjellige klassifiseringsmåter. Holder på FDIR sine.
df.drop(columns = ['Meldingsdato', 
                   'Meldingsklokkeslett', 
                   'Startdato',  
                   'Hovedområde start', 
                   'Lokasjon start (kode)', 
                   'Stoppdato', 
                   'Stoppklokkeslett', 
                   'Fangstår', 
                   'Hovedområde stopp (kode)', 
                   'Hovedområde stopp', 
                   'Lokasjon stopp (kode)', 
                   'Redskap FAO (kode)', 
                   'Redskap FAO', 
                   'Redskap FDIR', 
                   'Hovedart FAO (kode)', 
                   'Hovedart FAO', 
                   'Art FAO (kode)',
                   'Art FAO',
                   'Art - gruppe', 
                   'Lengdegruppe', 
                   'Bredde', 
                   'Fartøylengde',
                   'Bruttotonnasje 1969',
                   'Bruttotonnasje annen'
    ], inplace=True)

# Gruppere bruttotonnasje:
bins = [x for x in range(0, 4800, 100)] # Definerer gruppene
labels = range(1, len(bins))  # nummererer gruppene.
# Categorize 'Bruttotonnasje Kombinert' into the defined bins
df['Bruttotonnasje Gruppert'] = pd.cut(df['Bruttotonnasje Kombinert'], bins=bins, labels=labels)
# Teller grupperingene
df['Bruttotonnasje Gruppert'].value_counts()

Bruttotonnasje Gruppert
6     992
3     950
2     618
21    465
4     433
35    312
14    305
32    261
5     257
25    233
1     210
12    191
16    186
15    181
40    164
38     78
26     50
7      49
23     45
19     44
45     17
17      8
42      5
44      0
46      0
41      0
33      0
37      0
36      0
43      0
34      0
39      0
24      0
31      0
30      0
29      0
28      0
27      0
22      0
20      0
18      0
13      0
11      0
10      0
9       0
8       0
47      0
Name: count, dtype: int64

In [311]:
print(df.shape)
df.head()



(6054, 23)


,Melding ID,Meldingstidspunkt,Starttidspunkt,Startklokkeslett,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),Havdybde start,Stopptidspunkt,Varighet,...,Trekkavstand,Redskap FDIR (kode),Hovedart - FDIR (kode),Art - FDIR (kode),Art - FDIR,Art - gruppe (kode),Rundvekt,Lengdegruppe (kode),Bruttotonnasje Kombinert,Bruttotonnasje Gruppert
353,1497555,02.01.2018 14:01,01.01.2018 23:20,23:20,57.588,6.834,8.0,-229,02.01.2018 10:52,692,...,38303.0,51.0,2524.0,1022.0,Torsk,201.0,114.0,4.0,267.0,3
635,1497856,02.01.2018 23:54,02.01.2018 12:00,12:00,61.221,2.181,28.0,-167,02.01.2018 17:30,330,...,36931.0,51.0,1032.0,1022.0,Torsk,201.0,100.0,5.0,2053.0,21
1000,1498229,03.01.2018 19:15,03.01.2018 15:48,15:48,58.318,6.210,8.0,1,03.01.2018 16:59,71,...,1259.0,51.0,2524.0,1022.0,Torsk,201.0,10.0,3.0,79.0,1
1001,1498229,03.01.2018 19:15,03.01.2018 15:48,15:48,58.318,6.210,8.0,1,03.01.2018 16:59,71,...,1259.0,51.0,2524.0,1027.0,Hyse,202.0,8.0,3.0,79.0,1
1149,1498376,03.01.2018 23:37,02.01.2018 22:57,22:57,63.411,5.656,7.0,-356,03.01.2018 03:59,302,...,7558.0,51.0,1032.0,1022.0,Torsk,201.0,72.0,5.0,536.0,6


In [312]:
from sklearn.datasets import make_regression
x = df.drop('Rundvekt', axis=1)
y = df['Rundvekt']
X, y = make_regression(n_samples=200, random_state=1)

In [313]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
print(X)

[[0.53849796 0.67165789 0.55061365 ... 0.08717505 0.36709735 0.32678704]
 [0.27181601 0.53867371 0.80831695 ... 0.2761312  0.5138262  0.46871348]
 [0.34618015 0.45092554 0.57773963 ... 0.76975405 0.4525061  0.12122787]
 ...
 [0.74660208 0.66142304 0.24805112 ... 0.6689463  0.28902679 0.52068097]
 [0.30496389 0.78475825 0.72771027 ... 0.83227308 0.68756009 0.1908134 ]
 [0.01654068 0.24950367 0.64919125 ... 0.72556408 0.55990952 0.38590157]]


In [314]:
# X.shape = (6054, 23)
# y.shape = (6054,)

In [315]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 1235)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [316]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

scaler = sc.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_testscaled = scaler.transform(X_test)

In [317]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor(hidden_layer_sizes = [20,10], 
                  activation = 'logistic', alpha = 0.1, 
                  max_iter = 1000, #How many iterations of regression you do. higher = bigger chance of overtraining
                  solver = 'adam', 
                  verbose = False)
#activation logistic tanh relu identity
#solver adam lbfgs sgd 

In [318]:
nn.fit(X_train,y_train)
print("Neural network, MLPRegressor")
print("Test score",  nn.score(X_test, y_test))
print("Train  score", nn.score(X_train, y_train))

Neural network, MLPRegressor
Test score -0.08174727731627551
Train  score 0.07722199572211574


c:\Mashinelearning seminaroppgave\INFO284\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Without removed collumns
Test score 0.03306077219837722
Train  score 0.07260871122128132

with removed collumnds (linear regression collumns)
Test score 0.034291607331412455
Train  score 0.07651381759634102